In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import json
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from trainer import Trainer
from utils import Logger
print("Modules loaded")

Modules loaded


In [5]:
#This function gets the parameters from the config.json file 
def get_instance(module, name, config, *args):
    return getattr(module, config[name]['type'])(*args, **config[name]['args'])

def main2(config, resume):
    # setup data_loader instances
    data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size=4,
        shuffle=False,
        validation_split=0.0,
        training=False,
        num_workers=2
    )
    
    # build model architecture
    model = get_instance(module_arch, 'arch', config)
    print(model)
    if torch.cuda.is_available():
        print(torch.cuda.get_device_name(0))   
    else:
        print("using CPU")

    
    
    # get function handles of loss and metrics
    loss_fn = getattr(module_loss, config['loss'])
    metric_fns = [getattr(module_metric, met) for met in config['metrics']]
    
    # load state dict
    checkpoint = torch.load(resume, map_location='cpu')
    state_dict = checkpoint['state_dict'] #dictionary of model parameters from saved file
    if config['n_gpu'] > 1:
        model = torch.nn.DataParallel(model)
    model.load_state_dict(state_dict) 
    
    
    # prepare model for testing
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval() #tells model to ignore dropout and batch normalization
    
    
    heatmap(model, data_loader)
    
    
    total_loss = 0.0
    total_metrics = torch.zeros(len(metric_fns))
    
    with torch.no_grad(): #speed up calculations, unable to perform back propogation
        for i, (data, target) in enumerate(tqdm(data_loader)): #tqdm is a progress bar
            data, target = data.to(device), target.to(device)
            output = model(data)
            #
            # save sample images, or do something with output here
            #
            
            
            if i < 5:
                fig = plt.figure()
                output_cpu = output.to(torch.device("cpu"))
                plt.title("Prediction = " + str(np.argmax(output_cpu[1], axis=0)))
                data_cpu = data.to(torch.device("cpu"))
                plt.imshow(data_cpu[1].view([28,28]))
                
            # computing loss, metrics on test set
            loss = loss_fn(output, target)
            batch_size = data.shape[0]
            total_loss += loss.item() * batch_size
            for i, metric in enumerate(metric_fns):
                total_metrics[i] += metric(output, target) * batch_size
        
        plt.show()
                      
    n_samples = len(data_loader.sampler)
    print("num samples = " + str(n_samples))
    log = {'loss': total_loss / n_samples}
    log.update({met.__name__: total_metrics[i].item() / n_samples for i, met in enumerate(metric_fns)})
    print(log)
    print("My_metric is accuracy, my_metric2 is top-3 accuracy")
    
    
def heatmap(model, dataloader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    images, labels = next(iter(dataloader))
    images, labels = images.to(device), labels.to(device)
    preds = model(images)
    def hook_feature(module, inp, outp):
        features_blobs.append(outp.data.cpu().numpy())
        
    for image in images:
        layer_names = list(model._modules.keys())
        last_layer = layer_names[len(layer_names)-1]
        final_layer = model._modules.get(last_layer)
        features_blobs = []
        model._modules.get(final_layer).register_forward_hook(hook_feature)


    

    
    print("success")

In [6]:
resume = "saved/Mnist_LeNet/0414_203632/model_best.pth"

print(os.path.isdir("saved/Mnist_LeNet/0414_203632"))
print(os.path.exists(resume))

#config = torch.load("saved/Mnist_LeNet/0414_203632/config.json")
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

config_file = 'config.json'
# load config file
with open(config_file) as handle:
    config = json.load(handle)
# setting path to save trained models and log files
path = os.path.join(config['trainer']['save_dir'], config['name'])

True
True


In [7]:
#Test
main2(config, resume)

MnistModel(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Trainable parameters: 21840
using CPU


AttributeError: 'NoneType' object has no attribute 'register_forward_hook'